In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import tweepy
import io
import os
from dotenv import find_dotenv, load_dotenv
import time

In [2]:
df_tae = pd.read_csv('twitter-archive-enhanced.csv')

df_tae.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1606,685667379192414208,NaN,NaN,2016-01-09 03:40:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Marty. He has no idea what happened he...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/685667379...,9,10,Marty,NaN,NaN,pupper,NaN
1215,715009755312439296,NaN,NaN,2016-03-30 02:56:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Siba. She's remarkably mobile. Very sl...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715009755...,12,10,Siba,NaN,NaN,NaN,NaN
135,866450705531457537,NaN,NaN,2017-05-22 00:28:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jamesy. He gives a kiss to every other...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/866450705...,13,10,Jamesy,NaN,NaN,pupper,NaN
1871,675147105808306176,NaN,NaN,2015-12-11 02:56:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're presenting a group project and the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675147105...,10,10,NaN,NaN,NaN,NaN,NaN
203,853299958564483072,NaN,NaN,2017-04-15 17:32:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rumpole. He'll be your Uber driver thi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/853299958...,13,10,Rumpole,NaN,NaN,NaN,NaN


In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

page = requests.get(url)
df_img = pd.read_csv(io.StringIO(page.content.decode('utf-8')), sep='\t')

with open('image-predictions.tsv', 'wb') as file:
    file.write(page.content)

df_img.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1101,720785406564900865,https://pbs.twimg.com/media/CgC-gMCWcAAawUE.jpg,1,Chihuahua,0.896422,True,dingo,0.027929,False,kelpie,0.017916,True
547,677331501395156992,https://pbs.twimg.com/media/CWZdaGxXAAAjGjb.jpg,1,beagle,0.313464,True,boxer,0.218503,True,French_bulldog,0.106462,True
466,675015141583413248,https://pbs.twimg.com/media/CV4iqh5WcAEV1E6.jpg,1,street_sign,0.290091,False,golden_retriever,0.258372,True,sandbar,0.132173,False
1681,813812741911748608,https://pbs.twimg.com/media/C0s-XtzWgAAp1W-.jpg,1,French_bulldog,0.709146,True,Boston_bull,0.247621,True,boxer,0.018855,True
1397,768193404517830656,https://pbs.twimg.com/media/Cqkr0wiW8AAn2Oi.jpg,1,lion,0.396984,False,ram,0.300851,False,cheetah,0.094474,False


In [4]:
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
    
# Load credentials from environment variables
bearer_token_ = os.getenv('BEARER_TOKEN')
consumer_key_ = os.getenv('CONSUMER_KEY')
consumer_secret_ = os.getenv('CONSUMER_SECRET')
access_token_ = os.getenv('ACCESS_TOKEN')
access_secret_ = os.getenv('ACCESS_SECRET')

client = tweepy.Client(bearer_token=bearer_token_,
    consumer_key=consumer_key_,
    consumer_secret=consumer_secret_,
    access_token=access_token_,
    access_token_secret=access_secret_,
    wait_on_rate_limit=True,
    )

In [ ]:
tweet_ids = df_img['tweet_id'].tolist()

# Process in batches of 100 so it works for the get_tweets method
with open('tweet_json.txt', 'a', encoding='utf-8') as file:
    for i in range(0, len(tweet_ids), 100):
        try:
            tweets = client.get_tweets(tweet_ids[i:i+100], tweet_fields=['public_metrics','text'])
            if tweets.data:
                for tweet in tweets.data:
                    tweet_info = {
                        'tweet_id': tweet.id,
                        'text': tweet.text,
                        'retweet_count': tweet.public_metrics['retweet_count'],
                        'like_count': tweet.public_metrics['like_count'],
                        'reply_count': tweet.public_metrics['reply_count'],
                        'quote_count': tweet.public_metrics['quote_count']
                    }
                    file.write(json.dumps(tweet_info) + '\n')
            else:
                print(f"No data returned for batch {i}")
                
        except tweepy.TooManyRequests as e:
            print("Rate limit reached. Retrying after 15 mins and 5 secs...")
            time.sleep(15 * 61)  # Wait 15 minutes and 5 secs.

        except Exception as e:
            print(f"An error occurred: {e}")